In [1]:
import pandas as pd
import Feature as ft


def refactor_data(df):
    # Fill `attack_cat` to 'normal'
    df = fill_attack_cat(df)

    # Change port starting with '0x' to decimal
    df['sport'] = df['sport'].apply(hex_to_dec)
    df['dsport'] = df['dsport'].apply(hex_to_dec)

    # Fill 'ct_flw_http_mthd' to average of each feature
    for feature in ['ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd']:
        df = fill_http_ftp(df, feature)

    # Change data type based on the project appendix
    df = convert_feature_type(df, ft.object_, str)
    df = convert_feature_type(df, ft.integer_, int)
    df = convert_feature_type(df, ft.float_, float)
    df = convert_feature_type(df, ft.binary_, bool)

    # Factorize 4 features (proto, state, service, attack_cat)
    df = factorize_feature(df, ft.factorize_)



def factorize_feature(df, features):
    for feature in features:
        df[feature] = pd.factorize(df[feature])[0]
    return df


def fill_attack_cat(df):
    df['attack_cat'] = df.attack_cat.fillna(value='normal').apply(
        lambda x: x.strip().lower())
    return df


def hex_to_dec(port):
    if isinstance(port, str) and port.startswith('0x'):
        return int(port, 16)
    else:
        return port


def fill_http_ftp(df, feature):
    mean_val = df[feature].mean()
    df[feature].fillna(mean_val, inplace=True)
    return df


def convert_feature_type(df, features, dtype):
    for feature in features:
        df[feature] = df[feature].astype(dtype)
    return df


In [2]:
import pandas as pd

ORIGINAL_CSV = '../data/UNSW-NB15-BALANCED-TRAIN.csv'

df = pd.read_csv(ORIGINAL_CSV, encoding='ISO-8859-1', low_memory=False)
df = refactor_data(df)

TypeError: Could not convert 04   0    00       00   0 0 0            000 0000               0     001     0        0        00   0      000    0     0 0              0    0  0        01   00   0  0000       0  00   00             00      00     0              0 0  00   0  0             00 0    0 0  00 0          0 0 0 0  0 00 10      0  1       0              000   0 01   0    30   0 0     0 0 0   0  0 100      0      0  0       00        00   00     00 0     0      0  00     0   000                00000 00 0     0     1 0   0    0 0        0 0       0  0 0  10      0 0  0  0 0 0   00      0 00 0   00  0 0   0   0 0    0                 0   0 00   10 0    00     00   00  00    0  0   0  0 0  00      0   0  0 0   0 0       0 0  0   0     0     0 0   0     0 00  0000  00    0 0 0    0     000   0 0        0 0     1  00  0    0     00 001      0   0     1     0  0     00  0          0 000   0  0 0 0       0     0     0   0         0 0      0        0     0 0 0 1     00 0  0000 00       0